<a href="https://colab.research.google.com/github/sanazmohammadi62/AdvancedRegression/blob/main/Sanaz%20Mohammadi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import os
import datetime
import random as rn
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Fix the seed for reproducibility
np.random.seed(30)
rn.seed(30)
tf.random.set_seed(30)

# File paths
train_doc = np.random.permutation(open('train.csv').readlines())
val_doc = np.random.permutation(open('val.csv').readlines())

batch_size = 32  # you can experiment with this batch size
img_idx = [0, 29]  # using the first and last frame in a video for simplicity
input_shape = (64, 64, 3)  # Assuming input image shape

# Generator function
def generator(source_path, folder_list, batch_size):
    print('Source path =', source_path, '; batch size =', batch_size)

    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size

        for batch in range(num_batches):
            batch_data = np.zeros((batch_size, *input_shape))
            batch_labels = np.zeros((batch_size, 5))  # Assuming 5 classes

            for folder in range(batch_size):
                imgs = os.listdir(source_path + '/' + t[folder + (batch * batch_size)].split(';')[0])

                for idx, item in enumerate(img_idx):
                    image = load_img(source_path + '/' + t[folder + (batch * batch_size)].strip().split(';')[0] + '/' + imgs[item])
                    image = img_to_array(image)
                    image = tf.image.resize(image, input_shape[:2])
                    image = image / 255.0  # Normalize to [0, 1]

                    batch_data[folder, :, :, idx] = image

                batch_labels[folder, int(t[folder + (batch * batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

# Model definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')  # Assuming 5 classes
])

# Optimizer
optimiser = Adam()  # you can adjust the learning rate
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

# Training setup
train_generator = generator('train', train_doc, batch_size)
val_generator = generator('val', val_doc, batch_size)

curr_dt_time = datetime.datetime.now()
model_name = 'model_init_' + str(curr_dt_time).replace(' ', '').replace(':', ',') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

# Callbacks
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=0.001)
callbacks_list = [checkpoint, LR]

# Training configuration
num_train_sequences = len(train_doc)
num_val_sequences = len(val_doc)
num_epochs = 20  # Example number of epochs

if (num_train_sequences % batch_size) == 0:
    steps_per_epoch = num_train_sequences // batch_size
else:
    steps_per_epoch = (num_train_sequences // batch_size) + 1

if (num_val_sequences % batch_size) == 0:
    validation_steps = num_val_sequences // batch_size
else:
    validation_steps = (num_val_sequences // batch_size) + 1

# Model training
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator, validation_steps=validation_steps,
                    class_weight=None, workers=1, initial_epoch=0)


FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [1]:
pip install tensorflow